# Pre-Proceso (1)


Validación de los datos
* Revisión de valores anómalos
* Revisión de valores faltantes
* Inconsistencias


Definir SparkContext y SQLContext

In [1]:
#from pyspark import SparkContext
#sc = SparkContext()
#from pyspark.sql import SQLContext
#sqlContext=SQLContext(sc)


In [2]:
bd = sqlContext.read.format("com.databricks.spark.csv"
).option("header", "true"
).load("On_Time_On_Time_Performance_2016_12.csv", 
       inferSchema=True)

In [3]:

bd=bd.select('Year','Month','DayofMonth','DayOfWeek','CRSDepTime','UniqueCarrier', 'TailNum', 'ArrDelay','DepDelay','Origin','Dest','Distance','Cancelled', 'Diverted','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay') 

In [4]:
bd2 = bd.filter( bd.Origin.isin('ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') & bd.Dest.isin(
    'ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') )

sqlContext.registerDataFrameAsTable(bd2, "bd2")

## Validación

### Valores Anómalos

In [6]:
bd2.describe('ArrDelay').show()

+-------+------------------+
|summary|          ArrDelay|
+-------+------------------+
|  count|             30466|
|   mean|13.300695857677411|
| stddev| 53.50946738163747|
|    min|             -53.0|
|    max|            2028.0|
+-------+------------------+



In [7]:
bd2.describe().show()

+-------+------+-----+------------------+------------------+------------------+------------------+-----------------+-----------------+--------------------+--------------------+-----------------+-----------------+------------------+--------------------+------------------+
|summary|  Year|Month|        DayofMonth|         DayOfWeek|        CRSDepTime|          ArrDelay|         DepDelay|         Distance|           Cancelled|            Diverted|     CarrierDelay|     WeatherDelay|          NASDelay|       SecurityDelay| LateAircraftDelay|
+-------+------+-----+------------------+------------------+------------------+------------------+-----------------+-----------------+--------------------+--------------------+-----------------+-----------------+------------------+--------------------+------------------+
|  count| 30965|30965|             30965|             30965|             30965|             30466|            30501|            30965|               30965|               30965|        

Exportamos la tabla a Pandas para mejorar la visualización

In [8]:
bd2.describe().toPandas().set_index('summary').transpose()

summary,count,mean,stddev,min,max
Year,30965,2016.0,0.0,2016,2016
Month,30965,12.0,0.0,12,12
DayofMonth,30965,15.943484579363798,8.876012086360868,1,31
DayOfWeek,30965,4.008719522041014,1.9259616915158724,1,7
CRSDepTime,30965,1336.3809785241401,507.6969342028542,5,2359
ArrDelay,30466,13.300695857677411,53.50946738163747,-53.0,2028.0
DepDelay,30501,17.28562997934494,49.93901250742787,-21.0,2040.0
Distance,30965,932.5556596156952,547.1294895897886,236.0,2139.0
Cancelled,30965,0.015533667043436138,0.12366432835495891,0.0,1.0
Diverted,30965,5.813014694009365E-4,0.024103574850036478,0.0,1.0


### Inconsistencias

Variable Distance en función de los diferentes aeropuertos 

In [9]:
bd2.select('Origin','Dest','Distance').groupBy('Origin','Dest').mean().show()

+------+----+-------------+
|Origin|Dest|avg(Distance)|
+------+----+-------------+
|   ATL| ORD|        606.0|
|   LAS| LAX|        236.0|
|   ATL| LAS|       1747.0|
|   LAS| ATL|       1747.0|
|   ATL| LAX|       1946.0|
|   DEN| PHX|        602.0|
|   ORD| LAS|       1514.0|
|   LAX| DEN|        862.0|
|   DEN| DFW|        641.0|
|   ORD| LAX|       1744.0|
|   DFW| ORD|        802.0|
|   DEN| SFO|        967.0|
|   ORD| ATL|        606.0|
|   DFW| LAS|       1055.0|
|   LAS| DEN|        628.0|
|   DFW| LAX|       1235.0|
|   PHX| ORD|       1440.0|
|   DFW| ATL|        731.0|
|   ATL| DEN|       1199.0|
|   PHX| LAS|        255.0|
+------+----+-------------+
only showing top 20 rows



In [63]:
sqlContext.sql("select Origin, Dest, count(*) as n, mean(Distance) as mean, \
                min(Distance) as min , max(Distance) as max,  \
                max(Distance) - min(Distance) as dif, \
                stddev(Distance) as stdev \
                from bd2 \
                group by Origin, Dest" ).show()

+------+----+---+------+------+------+---+--------------------+
|Origin|Dest|  n|  mean|   min|   max|dif|               stdev|
+------+----+---+------+------+------+---+--------------------+
|   ATL| ORD|530| 606.0| 606.0| 606.0|0.0|                 0.0|
|   LAS| LAX|998| 236.0| 236.0| 236.0|0.0|1.237017211398351...|
|   ATL| LAS|368|1747.0|1747.0|1747.0|0.0|                 0.0|
|   LAS| ATL|366|1747.0|1747.0|1747.0|0.0|                 0.0|
|   ATL| LAX|541|1946.0|1946.0|1946.0|0.0|                 0.0|
|   DEN| PHX|680| 602.0| 602.0| 602.0|0.0|                 0.0|
|   ORD| LAS|299|1514.0|1514.0|1514.0|0.0|                 0.0|
|   LAX| DEN|578| 862.0| 862.0| 862.0|0.0|5.634507783805266...|
|   DEN| DFW|453| 641.0| 641.0| 641.0|0.0|                 0.0|
|   ORD| LAX|742|1744.0|1744.0|1744.0|0.0|                 0.0|
|   DFW| ORD|578| 802.0| 802.0| 802.0|0.0|                 0.0|
|   DEN| SFO|674| 967.0| 967.0| 967.0|0.0|                 0.0|
|   ORD| ATL|541| 606.0| 606.0| 606.0|0.

In [62]:
sqlContext.sql("select Origin, Dest, count(*) as n, mean(Distance) as mean, \
                min(Distance) as min , max(Distance) as max,  \
                max(Distance) - min(Distance) as dif, \
                stddev(Distance) as stdev \
                from bd2 \
                group by Origin, Dest" ).toPandas()

,Origin,Dest,n,mean,min,max,dif,stdev
0,ATL,ORD,530,606.0,606.0,606.0,0.0,0.000000e+00
1,LAS,LAX,998,236.0,236.0,236.0,0.0,1.237017e-14
2,ATL,LAS,368,1747.0,1747.0,1747.0,0.0,0.000000e+00
3,LAS,ATL,366,1747.0,1747.0,1747.0,0.0,0.000000e+00
4,ATL,LAX,541,1946.0,1946.0,1946.0,0.0,0.000000e+00
5,DEN,PHX,680,602.0,602.0,602.0,0.0,0.000000e+00
6,ORD,LAS,299,1514.0,1514.0,1514.0,0.0,0.000000e+00
7,LAX,DEN,578,862.0,862.0,862.0,0.0,5.634508e-14
8,DEN,DFW,453,641.0,641.0,641.0,0.0,0.000000e+00
9,ORD,LAX,742,1744.0,1744.0,1744.0,0.0,0.000000e+00


### Valores Faltantes

In [11]:
bd2.describe().toPandas().transpose()[0]

summary              count
Year                 30965
Month                30965
DayofMonth           30965
DayOfWeek            30965
CRSDepTime           30965
ArrDelay             30466
DepDelay             30501
Distance             30965
Cancelled            30965
Diverted             30965
CarrierDelay          8401
WeatherDelay          8401
NASDelay              8401
SecurityDelay         8401
LateAircraftDelay     8401
Name: 0, dtype: object

Valores faltantes en ArrDelay

In [12]:
bd2.select('ArrDelay').count()

30965

In [13]:
from pyspark.sql.functions import col
bd2.describe('Year','ArrDelay').filter(col("summary") == "count").collect()

[Row(summary='count', Year='30965', ArrDelay='30466')]

In [14]:
bd2.select('ArrDelay').count() - bd2.select('ArrDelay').dropna().count()

499

Los valores faltantes en ArrDelay son causados por vuelos cancelados o desviados

In [15]:
bd2.select('Cancelled','Diverted').groupBy().sum().show()

+--------------+-------------+
|sum(Cancelled)|sum(Diverted)|
+--------------+-------------+
|         481.0|         18.0|
+--------------+-------------+



In [16]:
bd3 = bd2.filter(bd2.Cancelled==0
).filter(bd2.Diverted==0)

In [17]:
bd3.describe().toPandas().transpose()[0]

summary              count
Year                 30466
Month                30466
DayofMonth           30466
DayOfWeek            30466
CRSDepTime           30466
ArrDelay             30466
DepDelay             30466
Distance             30466
Cancelled            30466
Diverted             30466
CarrierDelay          8401
WeatherDelay          8401
NASDelay              8401
SecurityDelay         8401
LateAircraftDelay     8401
Name: 0, dtype: object

Valores faltantes en Tipos de *Delay*

In [18]:
bd3.filter(bd3.ArrDelay<0).select('ArrDelay','CarrierDelay','WeatherDelay',
                                  'NASDelay','SecurityDelay','LateAircraftDelay').show(20)

+--------+------------+------------+--------+-------------+-----------------+
|ArrDelay|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+--------+------------+------------+--------+-------------+-----------------+
|    -7.0|        null|        null|    null|         null|             null|
|    -3.0|        null|        null|    null|         null|             null|
|    -3.0|        null|        null|    null|         null|             null|
|    -2.0|        null|        null|    null|         null|             null|
|    -2.0|        null|        null|    null|         null|             null|
|    -6.0|        null|        null|    null|         null|             null|
|    -9.0|        null|        null|    null|         null|             null|
|    -2.0|        null|        null|    null|         null|             null|
|   -16.0|        null|        null|    null|         null|             null|
|   -21.0|        null|        null|    null|         null|     

In [19]:
bd3.filter(bd3.ArrDelay>=0).select('ArrDelay','CarrierDelay','WeatherDelay',
                                  'NASDelay','SecurityDelay','LateAircraftDelay').show(20)

+--------+------------+------------+--------+-------------+-----------------+
|ArrDelay|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+--------+------------+------------+--------+-------------+-----------------+
|     0.0|        null|        null|    null|         null|             null|
|     7.0|        null|        null|    null|         null|             null|
|    13.0|        null|        null|    null|         null|             null|
|     0.0|        null|        null|    null|         null|             null|
|     2.0|        null|        null|    null|         null|             null|
|     1.0|        null|        null|    null|         null|             null|
|    13.0|        null|        null|    null|         null|             null|
|    10.0|        null|        null|    null|         null|             null|
|     8.0|        null|        null|    null|         null|             null|
|    51.0|         0.0|         0.0|    51.0|          0.0|     

In [20]:
bd3.filter(bd3.ArrDelay==15).select('ArrDelay','CarrierDelay','WeatherDelay',
                                  'NASDelay','SecurityDelay','LateAircraftDelay').show(20)

+--------+------------+------------+--------+-------------+-----------------+
|ArrDelay|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+--------+------------+------------+--------+-------------+-----------------+
|    15.0|        15.0|         0.0|     0.0|          0.0|              0.0|
|    15.0|        15.0|         0.0|     0.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|     0.0|          0.0|             15.0|
|    15.0|         0.0|         0.0|     0.0|          0.0|             15.0|
|    15.0|         0.0|         0.0|     0.0|          0.0|             15.0|
|    15.0|        15.0|         0.0|     0.0|          0.0|     

In [21]:
bd3.filter(bd3.ArrDelay>=15).select('ArrDelay','CarrierDelay','WeatherDelay',
                                  'NASDelay','SecurityDelay','LateAircraftDelay').count()

8401

In [22]:
bd4 = bd3.na.fill({'CarrierDelay':0, 'WeatherDelay':0,'NASDelay':0,'SecurityDelay':0, 'LateAircraftDelay':0})
#bd4 = bd3.na.fill(0)
#bd4 = bd3.na.drop()

In [23]:
bd4.describe().toPandas().transpose()[0]

summary              count
Year                 30466
Month                30466
DayofMonth           30466
DayOfWeek            30466
CRSDepTime           30466
ArrDelay             30466
DepDelay             30466
Distance             30466
Cancelled            30466
Diverted             30466
CarrierDelay         30466
WeatherDelay         30466
NASDelay             30466
SecurityDelay        30466
LateAircraftDelay    30466
Name: 0, dtype: object